In [179]:
import pandas as pd


In [180]:
pm_10 = pd.read_csv('yurukov/sofia_airpollution.csv')

# Seasons

In [182]:
weather_raw = pd.read_csv("./data/sofia-weather-data.csv")

In [183]:
weather = weather_raw.dropna(axis=1, how='all')


In [184]:
weather.drop(axis=1,labels=['weather_icon'], inplace=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [185]:
weather["temp_cent"] = weather["temp"] - 272.15
weather["temp_max_cent"] = weather["temp_max"] - 272.15
weather["temp_min_cent"] = weather["temp_min"] - 272.15
weather['date'] = weather['dt_iso'].apply(lambda x : str.split(x, ' +')[0])
# f = {'temp_cent':['min','mean', 'max'], 'pressure':['mean'], 'humidity': ['mean'], 'wind_speed': ['min', 'mean', 'max']}

processed_weather = weather.groupby('date', as_index=False).mean()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [186]:
dates = pd.DataFrame(weather['date'].unique(), columns=["date"])

In [188]:
# weather = weather[weather['aaa'] != 1]
pm_10 = pm_10[pm_10['inv_PM10'] != 1]
pm_10 = pm_10[pm_10['inv_NO2'] != 1]

In [193]:
pm_10['timestamp'] = pm_10['timestamp'].astype("str")
processed_weather['date'] = processed_weather['date'].astype("str")

In [196]:
merged_with_date = pd.merge(processed_weather, pm_10, left_on='date', right_on="timestamp")

In [198]:
merged_with_date = merged_with_date.drop(axis=1, labels=['temp', 'dt', 'city_id', 'wind_deg', 'rain_1h', 'rain_3h',
       'rain_today', 'weather_id', 'inv_PM10', 'inv_NO2', 'NO', 'inv_NO', 'C6H6', 'inv_C6H6', 'CO',
       'inv_CO', 'O3', 'inv_O3', 'SO2', 'inv_SO2', 'HU', 'inv_HU', 'AP',
       'inv_AP', 'WND', 'inv_WND', 'SUN', 'inv_SUN', 'TMP', 'inv_TMP'])

In [201]:
#print(merged_with_date.columns)
#print(merged_with_date.head())
print(merged_with_date.describe())

           temp_min      temp_max      pressure      humidity    wind_speed  \
count  92791.000000  92791.000000  92791.000000  92791.000000  92791.000000   
mean     284.063221    284.063221   1007.293746     74.730076      2.599250   
std        9.933073      9.933073     27.712935     20.465179      2.058892   
min      252.150000    252.150000    920.000000     12.000000      0.000000   
25%      277.150000    277.150000   1011.000000     60.000000      1.000000   
50%      284.150000    284.150000   1016.000000     80.000000      2.000000   
75%      291.150000    291.150000   1021.000000     92.000000      4.000000   
max      312.150000    312.150000   1040.000000    100.000000     14.000000   

       rain_24h  snow_3h    clouds_all     temp_cent  temp_max_cent  \
count       0.0      0.0  92791.000000  92791.000000   92791.000000   
mean        NaN      NaN     37.339182     11.913221      11.913221   
std         NaN      NaN     36.741316      9.933073       9.933073   
min 

# Shift weather one day forward - for predictions

In [200]:
merged_with_date["temp_shifted"] = merged_with_date.temp_cent.shift(1)
merged_with_date["temp_min_shifted"] = merged_with_date.temp_min.shift(1)
merged_with_date["temp_max_shifted"] = merged_with_date.temp_max.shift(1)

#TODO